
# Pytorch + HuggingFace 
## KoElectra Model
박장원님의 KoElectra-small 사용<br>
https://monologg.kr/2020/05/02/koelectra-part1/<br>
https://github.com/monologg/KoELECTRA

## Dataset
네이버 영화 리뷰 데이터셋<br>
https://github.com/e9t/nsmc

## References
- https://huggingface.co/transformers/training.html
- https://tutorials.pytorch.kr/beginner/data_loading_tutorial.html
- https://tutorials.pytorch.kr/beginner/blitz/cifar10_tutorial.html
- https://wikidocs.net/44249

## 주의사항
꼭 GPU로 해주세요 - 1epoch 당 약 20분 소요

In [ ]:
# HuggingFace transformers 설치 및 NSMC 데이터셋 다운로드
!pip install transformers
!wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt
!wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt

     |████████████████████████████████| 2.0MB 14.4MB/s 
     |████████████████████████████████| 890kB 48.4MB/s 
     |████████████████████████████████| 3.2MB 50.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=3aaff066b7ca8ba700d3a23c386030ea97bca9e5efad74a959219c31ebe3c4ef
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
--2021-03-24 00:40:45--  https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4893335 (4.7M) [text/plain]
Saving to: ‘ratings_test.txt’

ratings_test.txt    100%[===================>]   4.67M  --.-KB/s    in 0.06s   

2021-03-24 00:40:45 (8

In [ ]:
!head ratings_train.txt
!head ratings_test.txt

id	document	label
9976970	아 더빙.. 진짜 짜증나네요 목소리	0
3819312	흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나	1
10265843	너무재밓었다그래서보는것을추천한다	0
9045019	교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정	0
6483659	사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다	1
5403919	막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.	0
7797314	원작의 긴장감을 제대로 살려내지못했다.	0
9443947	별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네	0
7156791	액션이 없는데도 재미 있는 몇안되는 영화	1
id	document	label
6270596	굳 ㅋ	1
9274899	GDNTOPCLASSINTHECLUB	0
8544678	뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아	0
6825595	지루하지는 않은데 완전 막장임... 돈주고 보기에는....	0
6723715	3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??	0
7898805	음악이 주가 된, 최고의 음악영화	1
6315043	진정한 쓰레기	0
6097171	마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가,고개를 젖게한다	0
8932678	갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한cg남무 아 그립다 동사서독같은 영화가 이건 3류아류작이다	0


In [ ]:
import pandas as pd
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, ElectraForSequenceClassification, AdamW
from tqdm.notebook import tqdm

In [ ]:
# GPU 사용
device = torch.device("cuda")

# Dataset 만들어서 불러오기 

In [ ]:
class NSMCDataset(Dataset):
  
  def __init__(self, csv_file):
    # Nan 제거
    self.dataset = pd.read_csv(csv_file, sep='\t').dropna(axis=0) 
    # 중복 제거
    self.dataset.drop_duplicates(subset=['document'], inplace=True)
    self.tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-small-v2-discriminator")

    print(self.dataset.describe())
  
  def __len__(self):
    return len(self.dataset)
  
  def __getitem__(self, idx):
    row = self.dataset.iloc[idx, 1:3].values
    text = row[0]
    y = row[1]

    inputs = self.tokenizer(
        text, 
        return_tensors='pt',
        truncation=True,
        max_length=256,
        pad_to_max_length=True,
        add_special_tokens=True
        )
    
    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]

    return input_ids, attention_mask, y

In [ ]:
train_dataset = NSMCDataset("ratings_train.txt")
test_dataset = NSMCDataset("ratings_test.txt")


                 id          label
count  1.461820e+05  146182.000000
mean   6.779186e+06       0.498283
std    2.919223e+06       0.499999
min    3.300000e+01       0.000000
25%    4.814832e+06       0.000000
50%    7.581160e+06       0.000000
75%    9.274760e+06       1.000000
max    1.027815e+07       1.000000
                 id         label
count  4.915700e+04  49157.000000
mean   6.752945e+06      0.502695
std    2.937158e+06      0.499998
min    6.010000e+02      0.000000
25%    4.777143e+06      0.000000
50%    7.565415e+06      1.000000
75%    9.260204e+06      1.000000
max    1.027809e+07      1.000000


# Create Model

In [ ]:
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-small-v2-discriminator").to(device)

# 한번 실행해보기
# text, attention_mask, y = train_dataset[0]
# model(text.unsqueeze(0).to(device), attention_mask=attention_mask.unsqueeze(0).to(device))

Some weights of the model checkpoint at monologg/koelectra-small-v2-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-small-v2-discriminator and are newly initialized

In [ ]:
model.load_state_dict(torch.load("model.pt"))

FileNotFoundError: ignored

In [ ]:
# 모델 레이어 보기
model

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(32200, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (embeddings_project): Linear(in_features=128, out_features=256, bias=True)
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_

# Learn

In [ ]:
epochs = 3
batch_size = 128

In [ ]:
optimizer = AdamW(model.parameters(), lr=1e-5)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)

In [ ]:
losses = []
accuracies = []

for i in range(epochs):
  total_loss = 0.0
  correct = 0
  total = 0
  batches = 0

  model.train()

  for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):
    optimizer.zero_grad()
    y_batch = y_batch.to(device)
    y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
    loss = F.cross_entropy(y_pred, y_batch)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

    _, predicted = torch.max(y_pred, 1)
    correct += (predicted == y_batch).sum()
    total += len(y_batch)

    batches += 1
    if batches % 100 == 0:
      print("Batch Loss:", total_loss, "Accuracy:", correct.float() / total)
  
  losses.append(total_loss)
  accuracies.append(correct.float() / total)
  print("Train Loss:", total_loss, "Accuracy:", correct.float() / total)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2074: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Batch Loss: 68.11420601606369 Accuracy: tensor(0.6248, device='cuda:0')
Batch Loss: 127.55483287572861 Accuracy: tensor(0.6889, device='cuda:0')
Batch Loss: 177.60042506456375 Accuracy: tensor(0.7236, device='cuda:0')
Batch Loss: 222.82263535261154 Accuracy: tensor(0.7454, device='cuda:0')
Batch Loss: 266.242399841547 Accuracy: tensor(0.7591, device='cuda:0')
Batch Loss: 307.594554156065 Accuracy: tensor(0.7695, device='cuda:0')
Batch Loss: 348.2937306165695 Accuracy: tensor(0.7770, device='cuda:0')
Batch Loss: 387.00195291638374 Accuracy: tensor(0.7841, device='cuda:0')
Batch Loss: 425.4674153625965 Accuracy: tensor(0.7894, device='cuda:0')
Batch Loss: 463.17021495103836 Accuracy: tensor(0.7942, device='cuda:0')
Batch Loss: 499.92631831765175 Accuracy: tensor(0.7983, device='cuda:0')

Train Loss: 515.8876439034939 Accuracy: tensor(0.7997, device='cuda:0')


Batch Loss: 35.17871482670307 Accuracy: tensor(0.8521, device='cuda:0')
Batch Loss: 70.73329131305218 Accuracy: tensor(0.8487, device='cuda:0')
Batch Loss: 106.20480132102966 Accuracy: tensor(0.8478, device='cuda:0')
Batch Loss: 141.82859934866428 Accuracy: tensor(0.8463, device='cuda:0')
Batch Loss: 176.77495354413986 Accuracy: tensor(0.8470, device='cuda:0')
Batch Loss: 211.19762663543224 Accuracy: tensor(0.8475, device='cuda:0')
Batch Loss: 244.8506855815649 Accuracy: tensor(0.8486, device='cuda:0')
Batch Loss: 278.6458894610405 Accuracy: tensor(0.8492, device='cuda:0')
Batch Loss: 311.38765136897564 Accuracy: tensor(0.8501, device='cuda:0')
Batch Loss: 345.38014464080334 Accuracy: tensor(0.8503, device='cuda:0')
Batch Loss: 378.7224003225565 Accuracy: tensor(0.8507, device='cuda:0')

Train Loss: 393.01246170699596 Accuracy: tensor(0.8508, device='cuda:0')


Batch Loss: 33.14909455180168 Accuracy: tensor(0.8560, device='cuda:0')
Batch Loss: 66.06335058808327 Accuracy: tensor(0.8571, device='cuda:0')
Batch Loss: 98.10395792126656 Accuracy: tensor(0.8582, device='cuda:0')
Batch Loss: 129.00757251679897 Accuracy: tensor(0.8608, device='cuda:0')
Batch Loss: 161.1630758047104 Accuracy: tensor(0.8613, device='cuda:0')
Batch Loss: 192.5691910535097 Accuracy: tensor(0.8617, device='cuda:0')
Batch Loss: 223.94444207847118 Accuracy: tensor(0.8627, device='cuda:0')
Batch Loss: 254.5665091574192 Accuracy: tensor(0.8632, device='cuda:0')
Batch Loss: 285.7238003909588 Accuracy: tensor(0.8636, device='cuda:0')
Batch Loss: 316.6031837463379 Accuracy: tensor(0.8643, device='cuda:0')
Batch Loss: 347.06563840806484 Accuracy: tensor(0.8648, device='cuda:0')

Train Loss: 360.14852303266525 Accuracy: tensor(0.8649, device='cuda:0')


In [ ]:
losses, accuracies

([515.8876439034939, 393.01246170699596, 360.14852303266525],
 [tensor(0.7997, device='cuda:0'),
  tensor(0.8508, device='cuda:0'),
  tensor(0.8649, device='cuda:0')])

테스트 데이터셋 정확도 확인하기

In [ ]:
model.eval()

test_correct = 0
test_total = 0

for input_ids_batch, attention_masks_batch, y_batch in tqdm(test_loader):
  y_batch = y_batch.to(device)
  y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
  _, predicted = torch.max(y_pred, 1)
  test_correct += (predicted == y_batch).sum()
  test_total += len(y_batch)

print("Accuracy:", test_correct.float() / test_total)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2074: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,



Accuracy: tensor(0.8675, device='cuda:0')


In [ ]:
# 모델 저장하기
torch.save(model.state_dict(), "model.pt")